In [93]:
import cv2
import pandas as pd
import os

from glob import glob
from pyzbar.pyzbar import decode as pyzbar_decode

In [98]:
MFD_FILE = 'mfd_23_2_5206.xlsx'
SOURCE = 'test_image'
RESULT = 'result'

In [67]:
def decode_qr_code(image):
    # Initialize the QRCode detector
    qr_code_detector = cv2.QRCodeDetector()

    # Detect and decode the QR code
    data, vertices_array, _ = qr_code_detector.detectAndDecode(image)

    if vertices_array is not None:
        qr_location = "Detected"
    else:
        qr_location = "Not detected"

    return data, qr_location

def decode_qr_code_2(image):
    # Initialize the QRCode detector
    qr_code_detector = cv2.QRCodeDetector()

    # Detect and decode the QR code
    data, vertices_array, _ = qr_code_detector.detectAndDecode(image)

    if vertices_array is not None:
        qr_location = "Detected"
        vertices_array = vertices_array[0]

        for i in range(len(vertices_array)):
            pt1 = (
                int(vertices_array[i][0]),
                int(vertices_array[i][1])
            )
            pt2 = (
                int(vertices_array[(i + 1) % 4][0]),
                int(vertices_array[(i + 1) % 4][1])
            )
            cv2.line(image, pt1, pt2, (0, 255, 0), 3)

    else:
        qr_location = "Not detected"

    return data, qr_location

def rotate_image(image, angle):
    if angle == 180:
        return cv2.rotate(image, cv2.ROTATE_180)
    elif angle == 90:
        return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 270:
        return cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return image

In [68]:
def split_image_and_detect_qr(image_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if width > height:
        rows, cols = 4, 5
    else:
        rows, cols = 5, 4

    part_height = height // rows
    part_width = width // cols

    qr_data = {}
    rotate_angle = None

    for row in range(rows):
        for col in range(cols):
            part = image[row * part_height:(row + 1) * part_height, col * part_width:(col + 1) * part_width]
            part_key = f"part_{row + 1}_{col + 1}"
            data, qr_location = decode_qr_code(part)
            qr_data[part_key] = {"data": data, "location": qr_location}

            if qr_location == "Detected" and len(data) >= 10:
                if row == 4 and col == 3:
                    pass
                elif row == 0 and col == 0:
                    rotate_angle = 180
                elif row == 3 and col == 0:
                    rotate_angle = 270
                elif row == 0 and col == 4:
                    rotate_angle = 90
                elif row == 2 and col == 4:
                    pass
                elif row == 4 and col == 1:
                    rotate_angle = 270
                elif row == 1 and col == 0:
                    rotate_angle = 180
                elif row == 0 and col == 2:
                    rotate_angle = 90

    rotated_image = image
    if rotate_angle:
        rotated_image = rotate_image(image, rotate_angle)

    qr_code_data = ""
    for key, value in qr_data.items():
        if value["location"] == "Detected" and len(value["data"]) >= 10:
            qr_code_data = value["data"]
            break

    return qr_code_data, rotated_image

In [99]:
mfd = pd.read_excel(MFD_FILE, dtype=str)

In [100]:
for i in glob(f'{SOURCE}/*.jpg'):
    qr_code_data, rotated_image = split_image_and_detect_qr(i)
    if qr_code_data:
        fname = qr_code_data.split('\n')[0].split('_')[0]
        kec = mfd.loc[mfd['iddesa']==fname, 'nmkec'].values[0]
        desa = mfd.loc[mfd['iddesa']==fname, 'nmdesa'].values[0]

        folder = f'{RESULT}/{kec}/{desa}'
        if not os.path.exists(folder):
            os.makedirs(folder)

        cv2.imwrite(
            f'{folder}/{fname}.jpg',
            rotated_image
        )
    else:
        print(f"No QR code detected. {i}")